<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/beta/af/examples/hallucination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AlphaFold - Hallucination (beta version)
For a given length, generate/hallucinate a protein sequence that AlphaFold thinks folds into a well structured protein (high plddt, low pae, many contacts).

**WARNING**
1.   This notebook is in active development and was designed for demonstration purposes only.
2.   Using AfDesign as the only "loss" function for design might be a bad idea, you may find adversarial sequences (aka. sequences that trick AlphaFold).

In [ ]:
#@title install
%%bash
if [ ! -d af_backprop ]; then
  git clone https://github.com/sokrypton/af_backprop.git
  pip -q install biopython dm-haiku==0.0.5 ml-collections py3Dmol
fi
if [ ! -d params ]; then
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabDesign/beta/af/design.py
wget -qnc https://raw.githubusercontent.com/sokrypton/ColabDesign/beta/af/bkg_prob.txt


In [ ]:
#@title import libraries
import sys
sys.path.append('/content/af_backprop')

import os
from google.colab import files
import numpy as np
from IPython.display import HTML
from design import mk_design_model, clear_mem

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
#@title hallucinate
length =  100#@param {type:"integer"}
copies =  1

if "model" not in dir() or model._len != length or model._copies != copies:
  clear_mem()
  model = mk_design_model(protocol="hallucination")
  model.prep_inputs(length=length, copies=copies)

# pre-design with gumbel initialization and softmax activation
model.restart(seq_init="gumbel")
model.design(50, soft=True)

# three stage design  
model._state = model._init_fun({"seq":model._outs["seq_pseudo"]})
model.design_3stage(50,50,10)

In [ ]:
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb()

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()